# 🍎 건강 계산기 에이전트 튜토리얼 🍏

**건강 계산기 에이전트** 튜토리얼을 통해 건강 계산기를 사용하는 방법을 소개합니다:
1. 프로젝트를 **초기화**하고 Azure AI Foundry 사용하기
2. **코드 인터프리터** 기능을 사용하여 **에이전트 만들기**
3. 샘플 CSV 파일로 **BMI 계산** 및 **영양 데이터 분석** 수행
4. 기본 건강 인사이트 및 고지 사항 **생성**

> #### 이 노트북을 시작하기 전에 [`1-basics.ipynb`](./1-basics.ipynb) 노트북을 완료했는지 확인하세요.

## 시작하기
여기서는 **Fun & Fit** 샘플과 비슷하지만 숫자 계산과 데이터 분석에 **Code Interpreter**를 사용하는 데 중점을 두고 단계별로 안내해 드리겠습니다. 시작하겠습니다!

<img src="./seq-diagrams/2-code-interpreter.png" width="30%"/>




## 1. 초기 설정
라이브러리를 가져오고, 환경 변수를 로드하고, **AIProjectClient**를 초기화하는 것으로 시작하겠습니다. 데모를 위해 샘플 CSV도 생성하겠습니다.


In [ ]:
# Import required libraries
import os
import time
from pathlib import Path

import pandas as pd
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import CodeInterpreterTool, FilePurpose, MessageTextContent

# Load environment variables from the parent directory's .env
notebook_path = Path().absolute()
parent_dir = notebook_path.parent
load_dotenv(parent_dir / '.env')

# Initialize AIProjectClient
try:
    project_client = AIProjectClient.from_connection_string(
        credential=DefaultAzureCredential(),
        conn_str=os.environ["PROJECT_CONNECTION_STRING"],
    )
    print("✅ Successfully initialized AIProjectClient")
except Exception as e:
    print(f"❌ Error initializing client: {str(e)}")

# Create sample CSV data for demonstration
def create_sample_data():
    try:
        data = {
            'Date': pd.date_range(start='2024-01-01', periods=7),
            'Calories': [2100, 1950, 2300, 2050, 1900, 2200, 2150],
            'Protein_g': [80, 75, 85, 78, 72, 82, 79],
            'Carbs_g': [250, 230, 270, 245, 225, 260, 255],
            'Fat_g': [70, 65, 75, 68, 63, 73, 71],
            'Fiber_g': [25, 22, 28, 24, 21, 26, 23]
        }
        df = pd.DataFrame(data)
        filename = "nutrition_data.csv"
        df.to_csv(filename, index=False)
        print(f"📄 Created sample data file: {filename}")
        return filename
    except Exception as e:
        print(f"❌ Error creating sample data: {e}")
        return None

sample_file = create_sample_data()

## 2. 건강 계산기 에이전트 만들기 👩‍💻
샘플 CSV를 업로드한 다음 **코드 인터프리터**가 활성화된 에이전트를 생성합니다. 이 에이전트는 파일을 읽고, Python 코드를 실행하고, 결과와 시각화된 정보를 반환할 수 있습니다.


In [ ]:
def create_health_calculator(file_path):
    """Create an agent with code interpreter for health/nutrition calculations."""
    try:
        uploaded_file = project_client.agents.upload_file_and_poll(
            file_path=file_path,
            purpose=FilePurpose.AGENTS
        )
        print(f"✅ Uploaded CSV file, ID: {uploaded_file.id}")

        # Create a Code Interpreter tool referencing the uploaded file
        code_tool = CodeInterpreterTool(file_ids=[uploaded_file.id])

        # Create the agent with instructions to do basic calculations
        agent = project_client.agents.create_agent(
            model=os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4o-mini"),
            name="health-calculator-agent",
            instructions="""
            You are a health calculator agent that can:
            1. Calculate and interpret BMI
            2. Analyze provided nutrition data
            3. Generate charts/plots
            4. Include disclaimers that you are not a medical professional
            """,
            tools=code_tool.definitions,
            tool_resources=code_tool.resources
        )
        print(f"🎉 Created health calculator agent, ID: {agent.id}")
        return agent, uploaded_file
    except Exception as e:
        print(f"❌ Error creating health calculator agent: {e}")
        return None, None

health_agent, uploaded_file = None, None
if sample_file:
    health_agent, uploaded_file = create_health_calculator(sample_file)

## 3. 코드 인터프리터를 사용한 BMI 계산
BMI 계산을 위한 스레드를 만들겠습니다. 사용자의 키/몸무게를 입력하고 에이전트에게 BMI를 계산하는 방법을 보여주고 결과를 해석하며 항상 전문적인 조언을 거부하도록 요청합니다.


In [ ]:
def calculate_bmi_with_agent(agent, height_inches, weight_pounds):
    """Calculate BMI using the code interpreter agent."""
    try:
        # Create a new conversation thread
        thread = project_client.agents.create_thread()
        print(f"📝 Created thread for BMI calculation, ID: {thread.id}")

        # Construct user message requesting BMI calculation
        user_text = (
            f"Calculate BMI for \n"
            f"Height: {height_inches} inches\n"
            f"Weight: {weight_pounds} pounds\n"
            "Please: \n"
            "1. Show calculation \n"
            "2. Interpret the result \n"
            "3. Include disclaimers \n"
        )

        msg = project_client.agents.create_message(
            thread_id=thread.id,
            role="user",
            content=user_text
        )
        print(f"➕ Created BMI request message, ID: {msg.id}")

        # Create and process the run, letting the agent handle code
        run = project_client.agents.create_and_process_run(
            thread_id=thread.id,
            assistant_id=agent.id
        )
        print(f"🤖 BMI run finished with status: {run.status}")
        return thread, run
    except Exception as e:
        print(f"❌ Error during BMI calculation: {e}")
        return None, None

if health_agent:
    bmi_thread, bmi_run = calculate_bmi_with_agent(health_agent, 70, 180)  # example: 5'10" and 180 lbs


## 4. 영양 분석
사용자가 에이전트에게 업로드한 **`nutrition_data.csv`** 파일을 분석해 달라고 요청할 수 있는 또 다른 스레드를 만들겠습니다. 에이전트는 파일을 읽고, 매크로를 계산하고, 차트를 생성하고, 개인화된 의료 조언을 제공하지 않음을 고지할 수 있습니다.


In [ ]:
def analyze_nutrition_data(agent):
    """Ask the agent to analyze the uploaded nutrition data."""
    try:
        thread = project_client.agents.create_thread()
        print(f"📝 Created thread for nutrition analysis, ID: {thread.id}")

        user_text = (
            "Analyze the CSV file with daily nutrition data.\n"
            "1. Compute average daily macros (calories, protein, carbs, fat, fiber).\n"
            "2. Create a chart to show trends.\n"
            "3. Discuss any insights or disclaimers.\n"
        )

        msg = project_client.agents.create_message(
            thread_id=thread.id,
            role="user",
            content=user_text
        )
        print(f"➕ Created nutrition request message, ID: {msg.id}")

        run = project_client.agents.create_and_process_run(
            thread_id=thread.id,
            assistant_id=agent.id
        )
        print(f"🤖 Nutrition run finished with status: {run.status}")
        return thread, run
    except Exception as e:
        print(f"❌ Error analyzing nutrition data: {e}")
        return None, None

if health_agent:
    nutrition_thread, nutrition_run = analyze_nutrition_data(health_agent)

## 5. 결과 및 시각화 보기 📊
에이전트가 텍스트 인사이트, 고지 사항, 심지어 차트가 있는 이미지를 만들 수도 있습니다. 스레드에서 가져와 보겠습니다!


In [ ]:
def view_agent_responses(thread_id):
    try:
        messages = project_client.agents.list_messages(thread_id=thread_id)
        print("\n🔎 Agent Responses:")
        for msg in messages.data:
            if msg.role == "assistant" and msg.content:
                for c in msg.content:
                    if hasattr(c, "text"):
                        print("Response:", c.text.value, "\n")

        # If images were generated, let's save them
        for img in messages.image_contents:
            file_id = img.image_file.file_id
            outname = f"chart_{file_id}.png"
            project_client.agents.save_file(file_id=file_id, file_name=outname)
            print(f"🖼️ Saved image output: {outname}")

    except Exception as e:
        print(f"❌ Error viewing agent responses: {e}")

# Display BMI calculations
if bmi_thread and bmi_run:
    print("\n=== BMI Calculation Results ===")
    view_agent_responses(bmi_thread.id)

# Display nutrition analyses
if nutrition_thread and nutrition_run:
    print("\n=== Nutrition Analysis Results ===")
    view_agent_responses(nutrition_thread.id)

## 6. 정리 및 모범 사례
원하는 경우 에이전트 및 샘플 데이터를 제거할 수 있습니다. 프로덕션 환경에서는 반복 사용을 위해 보관할 수 있습니다.

### 모범 사례 요약
1. **데이터 처리** – 입력 데이터의 유효성을 검사하고, 누락된 값을 처리하고, 첨부 파일을 적절하게 관리합니다.
2. **계산** – 공식 단계, 면책 조항을 제공하고, 범위를 일반적인 건강으로 제한하고, 사용자가 의사가 아님을 상기시킵니다.
3. **시각화** – 차트는 교육용 데모용이라는 명확한 라벨과 면책 조항을 사용하세요.
4. **보안** – 사용량을 모니터링하고, 독점 데이터를 다루는 경우 코드 인터프리터에 대한 접근을 제한합니다.


In [ ]:
def cleanup_all():
    try:
        # Delete the uploaded CSV file from the service
        if 'uploaded_file' in globals() and uploaded_file:
            project_client.agents.delete_file(uploaded_file.id)
            print("🗑️ Deleted uploaded file from agent service.")

        # Delete the agent if we created one
        if 'health_agent' in globals() and health_agent:
            project_client.agents.delete_agent(health_agent.id)
            print("🗑️ Deleted health calculator agent.")

        # Delete local CSV file
        if 'sample_file' in globals() and sample_file and os.path.exists(sample_file):
            os.remove(sample_file)
            print("🗑️ Deleted local sample CSV file.")

    except Exception as e:
        print(f"❌ Error during cleanup: {e}")

cleanup_all()

# 축하합니다! 🎉
이제 **코드 인터프리터** 도구가 포함된 **건강 계산기 에이전트**를 사용할 수 있습니다:
- **BMI 계산** 을 수행하며 의사가 아님을 밝힙니다.
- 간단한 CSV 기반 영양 데이터를 **분석**하고 인사이트 + 차트를 생성합니다.
- 이미지(차트) 및 텍스트 기반 인사이트를 반환합니다.

#### 이제 [3-file-search.ipynb](3-file-search.ipynb) 으로 이동해봅시다.

행복하고 (건강한) 코딩되세요! 💪
